# Complex Workflow with AxisVM and VTK

In [ ]:
from axisvm.com.client import start_AxisVM
axvm = start_AxisVM(visible=True, daemon=True)

In [5]:
from axisvm import examples
import pyvista as pv
import numpy as np

from sigmaepsilon.mesh.tri.triutils import edges_tri
from sigmaepsilon.mesh.topo import unique_topo_data, detach

vtkpath = examples.download_stand_vtk()
mesh = pv.read(vtkpath).cast_to_unstructured_grid().extract_surface().cast_to_unstructured_grid()
coords = mesh.points / 1000
topo = np.array(mesh.cells_dict[5], dtype=int)

edges, edgeIDs = unique_topo_data(edges_tri(topo))
edges += 1
edgeIDs += 1

In [7]:
modelId = axvm.Models.New()
axm = axvm.Models.Item[modelId]
axm.Settings.EditingTolerance = -1
wdir = ""

from axisvm.com.tlb import RPoint3d
foo = lambda x : RPoint3d(x=x[0], y=x[1], z=x[2])
axm.BeginUpdate()
axm.Nodes.BulkAdd(list(map(foo, coords)))
axm.EndUpdate()

from axisvm.com.tlb import lgtStraightLine, RLineData

def gen_line(edge):
    return RLineData(
        NodeId1 = edge[0],
        NodeId2 = edge[1],
        GeomType = lgtStraightLine
    )
    
axm.BeginUpdate()
axm.Lines.BulkAdd(list(map(gen_line, edges)))
axm.EndUpdate()

from axisvm.com.tlb import vFront
axm.View = vFront
axm.FitInView()

from axisvm.com.tlb import ndcEuroCode
axm.Settings.NationalDesignCode = ndcEuroCode
matId = axm.Materials.AddFromCatalog(ndcEuroCode, "S 235")

from axisvm.com.tlb import RSurfaceAttr, lnlTensionAndCompression, \
    RResistancesXYZ, schLinear, stShell, RElasticFoundationXYZ, \
    RNonLinearityXYZ, RSurface
    
SurfaceAttr = RSurfaceAttr(
    Thickness=1,
    SurfaceType=stShell,
    RefZId=0,
    RefXId=0,
    MaterialId=matId,
    ElasticFoundation=RElasticFoundationXYZ(0, 0, 0),
    NonLinearity=RNonLinearityXYZ(lnlTensionAndCompression,
                                  lnlTensionAndCompression,
                                  lnlTensionAndCompression),
    Resistance=RResistancesXYZ(0, 0, 0),
    Charactersitics=schLinear)

def gen_surface(edges):
    return RSurface(
        N=3,
        LineIndex1 = edges[0],
        LineIndex2 = edges[1],
        LineIndex3 = edges[2],
        Attr = SurfaceAttr,
        DomainIndex = 0
    )
axm.BeginUpdate()
axm.Surfaces.BulkAdd(list(map(gen_surface, edgeIDs)))
axm.EndUpdate()


0

Select points of action

In [3]:
axvm.BringToFront()
nodes_f = axm.Nodes.select_IDs(msg='Select nodes where nodal loads are to be imposed!')
nodes_u = axm.Nodes.select_IDs(msg='Select nodes where displacement penalties are to be enforced!')

In [4]:
"""coords = axm.coordinates()
topo = axm.Surfaces.topology()"""

In [6]:
"""import tetgen
tet = tetgen.TetGen(coords, topo)
tet.tetrahedralize(order=1, mindihedral=10, minratio=1.1, quality=True)
grid = tet.grid
coords = np.array(grid.points).astype(float)
topo = grid.cells_dict[10].astype(int)"""

In [ ]:
from sigmaepsilon.solid import Structure, PointData, FemMesh
from sigmaepsilon.mesh.space import StandardFrame
from sigmaepsilon.solid.fem.cells import TET4 as CellData
from sigmaepsilon.math.array import repeat

GlobalFrame = StandardFrame(dim=3)

# essential bc
fixity = np.zeros((coords.shape[0], 6), dtype=bool)
fixity[nodes_u, :3] = True
fixity[:, 3:] = True

# natural bc
F = 10
loads = np.zeros((coords.shape[0], 6))
loads[nodes_f, 2] = True

# pointdata
pd = PointData(coords=coords, frame=GlobalFrame,
               loads=loads, fixity=fixity)

# celldata
frames = repeat(GlobalFrame.show(), topo.shape[0])
cd = CellData(topo=topo, frames=frames)

# set up mesh and structure
E = 12000.
nu = 0.2
A = np.array([
    [1, nu, nu, 0, 0, 0], 
    [nu, 1, nu, 0, 0, 0],
    [nu, nu, 1, 0, 0, 0], 
    [0., 0, 0, (1-nu)/2, 0, 0],
    [0., 0, 0, 0, (1-nu)/2, 0],
    [0., 0, 0, 0, 0, (1-nu)/2]]) * (E / (1-nu**2))

mesh = FemMesh(pd, cd, model=A, frame=GlobalFrame)

structure = Structure(mesh=mesh)